In [180]:
import os
import re
import pandas as pd
import unicodedata
import time

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled, NoTranscriptAvailable
from datetime import datetime

In [44]:
#get authentification to make requests to youtube
#info about client_secret.json - https://developers.google.com/youtube/v3/quickstart/python#step_1_set_up_your_project_and_credentials

def auth_yt():
    scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "client_secret.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    return youtube

In [84]:
#retrieve metadata from playlist

def get_metas_from_pl(youtube, pl_id):
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        maxResults=50, #50 is upper limit
        playlistId=pl_id
    )
    response = request.execute()

    return response['items']

In [70]:
def get_first_comment(youtube, video_id):
    results = youtube.commentThreads().list(
        part="snippet",
        maxResults=1,
        videoId=video_id,
        textFormat="plainText"
    ).execute()

    return results["items"][0]

In [2]:
#convert a time-like string to float repr of secs
# e.g 45:01 -> 2701.0
#     1:03:30 -> 3810.0

def str_time_to_float_secs(st):
    nums = st.split(':')
    if len(nums) > 2:
        return float(nums[0]) * 3600 + float(nums[1]) * 60 + float(nums[2])
    else:
        return float(nums[0]) * 60 + float(nums[1])

In [1]:
#remove additional symbols from time-like string

def clear_time(st):
    time_pat = re.compile(r'(\d+:)+\d+')
    return re.search(time_pat, st).group(0)

#extract times and descriptions of timestamps from comment text

def get_timestamps(comment_text):
    ts_pat = re.compile(r'(\d+:)+\d+.+')
    times = []
    descs = []

    for line in comment_text.split('\n'):
        if re.match(ts_pat, line) is not None:
            #deal with unneccessary unicode symbols
            line = unicodedata.normalize('NFKD', line)
            words = line.split(' ', 1)
            time = str_time_to_float_secs(clear_time(words[0]))
            times.append(time)
            descs.append(words[1])
    return times, descs

In [46]:
yt = auth_yt()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=702629732282-al64n9mnq1qs4d894r0rt7omhpfiudht.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=A4mJgVgPfwRHGf0rj0c6xT3ekGVcaX&prompt=consent&access_type=offline
Enter the authorization code: 4/zAETjijBX6YWbdXc3yWo9iSt5xyWJIqAWzUJw70-i1zrCkVv1zGSCNk


In [208]:
#list of playlist ids
pl_ids = ["PLLkqhqUNdkjCC43di6igZ80NUJxFFfZNa"]

#for pl_id in pl_ids:
pl_id = pl_ids[0]

metas = get_metas_from_pl(yt, pl_id)

#get all video ids from playlist metainfo
video_ids = [meta['contentDetails']['videoId'] for meta in metas]

#list of rows to be converted to dataframe, every row is a dict
rows = []

for video_id in video_ids:
    print(video_id)
    try:
        #get subtitles
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ru'])
        #request and extract comment
        raw_comment = get_first_comment(yt, video_id)
        comment_text = raw_comment["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
        times, descs = get_timestamps(comment_text)
        #number of sentence in subtitles
        sent_num = 0
        #counter for timestamps
        ts_num = 0
        #padding for subtitle-timestamp correspondence
        eps = 0.01
        
        for sub in transcript:
                is_ts = 0
                desc = ''
                #start of subtitle
                start_t = sub['start']
                #duration of subtitle
                dur_t = sub['duration']
                
                if  (ts_num < len(times)) and (start_t + eps >= times[ts_num]):
                    desc = descs[ts_num]
                    is_ts = 1
                    ts_num += 1

                row_dict = {'video_id' : video_id,
                            'sentence_num' : sent_num,
                            'text' : sub['text'],
                            'time_start' : start_t,
                            'time_duration' : dur_t,
                            'is_timestamp' : is_ts,
                            'desc' : desc}

                rows.append(row_dict)
                sent_num += 1
    except (TranscriptsDisabled, NoTranscriptAvailable):
        print("Transcripts disabled in video " + video_id)
        
df = pd.DataFrame(rows)    

BCIurE0kubE
GnEPQy-ZiHI
QSN2JQBMM-A
Transcripts disabled in video QSN2JQBMM-A
lWd5FiHKCQE
Hg2Fc9yH0wk
JdtTrvp4Zu0
F4wlXIx9koI
k8FsA_9BB1A
OSx7BMWiU5c
BE_rqguxWlI
Transcripts disabled in video BE_rqguxWlI
BH9GtmGsRZw
U-m5cBgsLA8
d64ZcV9RDP4
23bLhZJkZqc
VMG0FgnRLBg
ulNQxL_f4DE
Ut0cju7hZjU
ApokyLNvSXk
EkxfbhP0HMg
3H0TU10TkLo
SU4LLbSE4Kg
2O-G1cjAnco
WtDcKY10eKw
5OFCJBVpz-E
z0XWByGcb0g
rTEJ_Z-9t7g
GhO6CqTTV3A
_RWJx5G8g5w
FEC7RYjsXDw
hN1XxpePOao
qX9oxpDIF3g
hj70Yt6vWC0
F1zdWlaIflM
fnBk_KiPnfk
XkpZpflbthg
A2FQIH4UlP0
jYnMbAIzcGc
HIbAhkegz14
uxL8QY5p-vk
en-09JJjas4
BsOlazpCQe0
h9RhL5ZRYS4
qXQYd9y0neU
z6CW1GDdLxY
bCqFVr15Wg0
LsJkJWn0_JM
Transcripts disabled in video LsJkJWn0_JM
NJrB1vffKVM
SLp_ze6zG3Y
StYQPNnpHLY
tT58wAkNMSU


In [209]:
#sanity check
print(df[df['is_timestamp'] == 1][['desc','text']])
df.to_csv('Zhiza_df.csv', index=False)

                                              desc  \
18                         На работе не укачивает?   
23     Что делать, если забыл выключить утюг дома?   
45                               Чем вы питаетесь?   
60                         Зачем закрывать туалет?   
71                       Плацкарт придумал дьявол?   
...                                            ...   
25291       Где и как начинать заниматься спортом?   
25305        Сколько может зарабатывать спортсмен?   
25309           Что нравится в своём виде спорта?   
25315              Что бесит в своём виде спорта?   
25339                   Закончи блиц тремя словами   

                                           text  
18       иногда страдает на самом деле это дело  
23      позвонить соседям испугаться дело в том  
45        ну если бы раньше бы я бы сказала что  
60               раньше их закрывали когда были  
71                   рай от но это что-то вроде  
...                                         ...  
2

In [202]:
#single video DEBUG

video_id = 'M8dcZpf05jE'
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ru'])

raw_comment = get_first_comment(yt, video_id)
comment_text = raw_comment["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
print(comment_text)
times, descs = get_timestamps(comment_text)
#print(times)
#print(transcript)
sent_num = 0
ts_num = 0
eps = 0.01

rows = []
for sub in transcript:
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ru'])
    raw_comment = get_first_comment(yt, video_id)
    comment_text = raw_comment["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
    times, descs = get_timestamps(comment_text)
    sent_num = 0
    ts_num = 0
    eps = 0.01

    for sub in transcript:
            is_ts = 0
            desc = ''
            start_t = sub['start'] 
            dur_t = sub['duration']
            if  (ts_num < len(times)) and (start_t + eps >= times[ts_num]):
                desc = descs[ts_num]
                is_ts = 1
                ts_num += 1

            row_dict = {'video_id' : video_id,
                        'sentence_num' : sent_num,
                        'text' : sub['text'],
                        'time_start' : start_t,
                        'time_duration' : dur_t,
                        'is_timestamp' : is_ts,
                        'desc' : desc}

            rows.append(row_dict)
            sent_num += 1
df_test = pd.DataFrame(rows)
print(df_test[df_test['is_timestamp'] == 1])

0:23 Сниматься в настоящем крематории в гробу — как это вообще? 
2:08 Как выглядят работники крематория? 
3:46 Сыграть член. Как, зачем и почему? 
7:29 Тут — старые прически Дудя. Спойлер: с ними все очень плохо
9:52 Почему Колокольников переехал в Канаду? 
12:29 Голливуд - как он там оказался? 
14:25 Сначала Юрий работал официантом. После армянской свадьбы пришлось уволиться 
15:50 Какой у него был псевдоним в начале карьеры? 
18:36 В чем гениальность Кирилла Серебренникова? 
20:00 Колокольников vs Маяковский
22:15 Какой у него паспорт: российский или американский? 
23:00 Почему даже после «Игры Престолов» он все равно играет только русских? Как выглядит схема поиска актеров на роль? 
27:30 Колокольников дает интервью Дудю на английском
28:38 Как обычно проходят кастинги в Голливуде? 
31:39 Каких русских режиссеров знают в США?
32:50. Из всех режиссеров в мире с кем бы он хотел поработать больше всего? 
33:34 Почему в американских фильмах столько косяков и стереотипов о русских? 
36:0